In [2]:
import tweepy
import pandas as pd
import time 
import csv
import numpy as np
from pathlib import Path

In [3]:
# create client tweepy
bearer_token = "AAAAAAAAAAAAAAAAAAAAAHqVaQEAAAAAQeqIjAve3PYwMwqZagLUY0MTapI%3D9qBwaEmiTtt1uWBX6Nnrnz71XibxrwsUHdEpZbzlaO4laVxeiI"
client = tweepy.Client(bearer_token)

In [4]:
fileCsv = Path("data/test-energi-baru-terbarukan.csv")
if fileCsv.is_file():
    df = pd.read_csv('data/test-energi-baru-terbarukan.csv')
    df = df.drop(columns=['Unnamed: 0'])
else:
    column_names = ["tweet_id","author_id","author_name","author_username","author_location","content","content_clean","like_count","retweet_count","reply_count","source","keyword","address","longitude","latitude","province","city","country","created_at"]
    df = pd.DataFrame(columns = column_names)
    df

In [6]:
keyword = "energi baru terbarukan -is:retweet lang:id"
search_df = pd.DataFrame()
loop = 1
loop_conditional = False
token = None
start_time = '2012-01-01T00:00:00Z'
end_time = '2022-06-30T00:00:00Z'

# initial search to get next_token or pagination
search = client.search_recent_tweets(
    query = keyword,
    tweet_fields=['public_metrics', 'author_id', 'created_at', 'source', 'text', 'geo'], 
    user_fields = ['name','username','location','verified'],
    expansions = ['geo.place_id', 'author_id'],
    place_fields = ['country','country_code'],
    # start_time=start_time,
    # end_time=end_time,
    max_results=10, 
    # next_token=token,
)

In [7]:
# save result of search to dataframe
for row in search.data:
    new_array = {}
    new_array["tweet_id"] = row['id'] 
    new_array["author_id"] = row['author_id'] 
    new_array["author_name"] = "" 
    new_array["author_username"] = "" 
    new_array["author_location"] = "" 
    new_array["content"] = row['text'] 
    new_array['content_clean'] = ""
    new_array["like_count"] = row['public_metrics']['like_count'] 
    new_array["retweet_count"] = row['public_metrics']['retweet_count'] 
    new_array["reply_count"] = row['public_metrics']['reply_count'] 
    new_array["source"] = row['source'] 
    new_array["keyword"] = "energi baru terbarukan" 
    new_array["address"] = "" 
    new_array["longitude"] = ""
    new_array["latitude"] = ""
    new_array["province"] = ""
    new_array["city"] = ""
    new_array["country"] = ""
    new_array["created_at"] = row['created_at'] 
    temp_data = pd.json_normalize(new_array ,  sep = "_")
    search_df = pd.concat([search_df, temp_data], ignore_index=True)  
    loop+=1

In [63]:
for row in search.includes['users']:
    search_df.loc[search_df["author_id"] == row.id, "author_username"] = row.username
    search_df.loc[search_df["author_id"] == row.id, "author_name"] = row.name
    search_df.loc[search_df["author_id"] == row.id, "author_location"] = row.location
    search_df.loc[search_df["author_id"] == row.id, "author_name"] = row.name

In [64]:
df = df.append(search_df, ignore_index=True)

/var/folders/ql/17t5tp294714_4dzbm4dmz640000gn/T/ipykernel_20211/652426727.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(search_df, ignore_index=True)


In [65]:
df = df.drop_duplicates(subset=['tweet_id'])

In [66]:
df.head(5)

,tweet_id,author_id,author_name,author_username,author_location,content,content_clean,like_count,retweet_count,reply_count,source,keyword,address,longitude,latitude,province,city,country,created_at
0,1559836556067848192,1219063624699768832,Ｌｉｌｉ🇲🇨,LiliAngelica6,indonesia,"Selanjutnya, dilakukan percepatan pembangunan ...",,0,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-17 09:36:29+00:00
1,1559608295656419329,1550408525146234880,Mustika Alvianingrum,MustikaAlviani1,None,@energitodayID @pertamina @KemenBUMN Indonesia...,,0,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-16 18:29:28+00:00
2,1559594369606643712,1550410058881974272,Viniatry Nurullah,ViniatryNurull1,None,@energitodayID @pertamina @KemenBUMN Indonesia...,,0,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-16 17:34:08+00:00
3,1559583994265890818,1553320522410528771,Cardito Naenggolan,CarditoNaenggo1,None,@energitodayID @pertamina @KemenBUMN Indonesia...,,0,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-16 16:52:54+00:00
4,1559579364085248000,1554345230056378368,Wildan Wardhani,Wardhani1Wildan,None,@energitodayID @pertamina @KemenBUMN Indonesia...,,0,0,0,Twitter Web App,energi baru terbarukan,,,,,,,2022-08-16 16:34:30+00:00


In [67]:
df.to_csv('data/test-energi-baru-terbarukan.csv')